### Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


### Drop Null target records

In [3]:
model_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [4]:
# Drop Null values of Target 
df = model_data.dropna(subset=['60_in_15mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

### Split the Time interval

In [5]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [6]:
model_df1.shape, oot_df1.shape

((101984, 2831), (17343, 2831))

In [28]:
features_list = [
# banking
'avg_daily_balance_last_1_months',
'max_balance_M1',
'min_balance_last_1_months',
'min_balance_M1',
'balance_5th_day_M2',
'balance_3th_day_M1',
'bounced_transactions_count',
'days_balance_lt_5000_M1',
'days_balance_lt_10000_M1',
'Min_Balance_Min_M1_M2_M3',
'num_credit_gt_salary',
'upi_trans_count',
'Max_Credit_Avg_M1_M2_M3',
'ratio_total_debit_to_total_credit_M3',
'cash_withdrawal_sum',
'num_debit_card_trans_last_2_months',
'ratio_active_loan_vs_salary',    
#new banking features
'ratio_balance_after_2_days_max_credit_M1',
'ratio_balance_after_7_days_max_credit_M2',
'diff_btw_stated_salary_vs_actual_salary',
'ratio_5th_day_balance_to_max_credit_M3',
                 

'month_year',
'60_in_15mob']

In [29]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [30]:
oot_df = oot_df1[features_list]

In [31]:
model_df.shape, oot_df.shape

((101984, 23), (17343, 23))

In [32]:
model_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    90856
1.0    11128
Name: count, dtype: int64

In [33]:
oot_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    15505
1.0     1838
Name: count, dtype: int64

In [34]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [35]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [36]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3032429/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [37]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3032429/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [38]:
X_oot.shape,y_oot.shape

((17343, 21), (17343,))

### Split Model dataset into Train and Test

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [40]:
X_train.shape, X_test.shape

((61190, 21), (40794, 21))

In [41]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(48952, 21) (12238, 21) (40794, 21)


In [42]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [43]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'lambda': 5.2,
    'alpha': 4.5,
    'gamma': 4.8,
    'scale_pos_weight':4.0,
    'random_state': 42
    
}


evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 800,
    early_stopping_rounds= 30,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.54076	eval-auc:0.54447
[10]	train-auc:0.55261	eval-auc:0.54052
[20]	train-auc:0.55349	eval-auc:0.54529
[30]	train-auc:0.55823	eval-auc:0.54610
[31]	train-auc:0.55696	eval-auc:0.54639


In [45]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.56, Gini: 0.12, KS: 0.08
Eval AUC: 0.55, Gini: 0.10, KS: 0.07
Test AUC: 0.55, Gini: 0.10, KS: 0.06
OOT AUC: 0.55, Gini: 0.11, KS: 0.08


In [46]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.56, Gini: 0.1164, KS: 0.0764
Test AUC: 0.55, Gini: 0.0971, KS: 0.0609
OOT AUC: 0.55, Gini: 0.1091, KS: 0.0761


In [47]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_Banking_Train_Pred.csv',index=False)
Eval.to_csv('LeoPayu_Banking_Eval_Pred.csv',index=False)
Test.to_csv('LeoPayu_Banking_Test_Pred.csv',index=False)
Oot.to_csv('LeoPayu_Banking_Oot_Pred.csv',index=False)


In [48]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                     Feature  Importance
5                 bounced_transactions_count        22.0
10                       cash_withdrawal_sum        18.0
6                   Min_Balance_Min_M1_M2_M3        15.0
7                       num_credit_gt_salary         8.0
1                  min_balance_last_1_months         7.0
2                             min_balance_M1         7.0
4                         balance_3th_day_M1         5.0
3                         balance_5th_day_M2         5.0
0                             max_balance_M1         3.0
13   diff_btw_stated_salary_vs_actual_salary         3.0
9       ratio_total_debit_to_total_credit_M3         2.0
8                    Max_Credit_Avg_M1_M2_M3         1.0
11        num_debit_card_trans_last_2_months         1.0
12  ratio_balance_after_2_days_max_credit_M1         1.0
14    ratio_5th_day_balance_to_max_credit_M3         1.0


In [49]:
feature_importance_df

Feature  Importance
5                 bounced_transactions_count        22.0
10                       cash_withdrawal_sum        18.0
6                   Min_Balance_Min_M1_M2_M3        15.0
7                       num_credit_gt_salary         8.0
1                  min_balance_last_1_months         7.0
2                             min_balance_M1         7.0
4                         balance_3th_day_M1         5.0
3                         balance_5th_day_M2         5.0
0                             max_balance_M1         3.0
13   diff_btw_stated_salary_vs_actual_salary         3.0
9       ratio_total_debit_to_total_credit_M3         2.0
8                    Max_Credit_Avg_M1_M2_M3         1.0
11        num_debit_card_trans_last_2_months         1.0
12  ratio_balance_after_2_days_max_credit_M1         1.0
14    ratio_5th_day_balance_to_max_credit_M3         1.0

In [50]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
5          0.158088         0.346168        0.371593        0.335170   4896   
4          0.132999         0.333942        0.335169        0.332651   1594   
3          0.108702         0.332053        0.332650        0.323467  32658   
2          0.090009         0.319050        0.323466        0.315170   4444   
1          0.064552         0.313731        0.315153        0.311464   5360   

        label_good  label_bad  
Decile                         
5             4122        774  
4             1382        212  
3            29108       3550  
2             4044        400  
1             5014        346

In [51]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
5          0.161765         0.345950        0.371593        0.335252   4080   
4          0.125777         0.334026        0.335252        0.332651   1447   
3          0.109201         0.332067        0.332650        0.323540  27106   
2          0.094443         0.319184        0.323535        0.315170   3653   
1          0.081189         0.313738        0.315153        0.311464   4508   

        label_good  label_bad  
Decile                         
5             3420        660  
4             1265        182  
3            24146       2960  
2             3308        345  
1             4142        366

In [52]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
5          0.156160         0.345778        0.372388        0.335136   1729   
4          0.101836         0.333949        0.335118        0.332651    599   
3          0.107492         0.332119        0.332650        0.323707  11545   
2          0.083067         0.319342        0.323687        0.315255   1565   
1          0.071391         0.313877        0.315153        0.311464   1905   

        label_good  label_bad  
Decile                         
5             1459        270  
4              538         61  
3            10304       1241  
2             1435        130  
1             1769        136

In [53]:
#Save the model
xgb_model.save_model('LeoPayu_Banking_60_in_15mob.json')
print('Model Saved')

Model Saved
